# Day 09 - o1

In [1]:
def main():
    # Read the input line
    with open('input.txt', 'r') as f:
        line = f.readline().strip()

    # The input line encodes alternating sequences: file_length, free_length, file_length, free_length, ...
    # We need to decode this into a disk layout of file blocks and free spaces.
    # Let's parse the input line into pairs of (file_length, free_length, file_length, ...).
    # Note: The problem statement implies the pattern always starts with a file length and alternates.
    # If there's an odd number of digits, that means it ends with a file length and no trailing free section.
    # But the examples show a pattern always: file_length, free_length, file_length, free_length, ...
    # We'll assume that either the line has even length or if it ends in a file length, that just means no trailing free space.

    digits = list(map(int, line))
    # We'll construct a structure:
    # files_and_frees = [(file_length_0, free_length_0), (file_length_1, free_length_1), ...]
    # But careful: If the number of digits is even: we have pairs: file_length, free_length. If odd: last is file_length only.
    # Actually, the problem states the digits alternate between file and free space lengths.
    # Let's just walk through the digits, alternating interpretation.
    file_lengths = []
    free_lengths = []
    file_next = True
    for d in digits:
        if file_next:
            file_lengths.append(d)
        else:
            free_lengths.append(d)
        file_next = not file_next

    # It's possible that free_lengths has one less element if the input ended on a file length.
    # Handle that by appending a 0 if needed.
    if len(free_lengths) < len(file_lengths):
        free_lengths.append(0)

    # Now we have parallel lists file_lengths and free_lengths.
    # Construct the actual disk layout.
    # Each file is assigned an ID based on order of files encountered.
    # Example: For file_lengths: [1,3,5], the IDs will be 0, 1, 2 respectively.
    # We'll build a list of characters: each file block is represented by its file ID (as a single digit/character),
    # and free spaces by '.'.

    disk = []
    file_id = 0
    for f_len, fs_len in zip(file_lengths, free_lengths):
        # Add f_len blocks of file_id
        for _ in range(f_len):
            disk.append(str(file_id))
        # Add fs_len blocks of free space
        for _ in range(fs_len):
            disk.append('.')
        file_id += 1

    # Now we need to perform the compacting process:
    # "move file blocks one at a time from the end of the disk to the leftmost free space block, until no gaps remain."
    # A gap means there's a '.' followed at some position later by a file block.
    # The method:
    # While there exists a '.' that occurs before any file block to its right:
    #   Find the leftmost '.' and the rightmost file block that occurs after that '.' and move it.
    #
    # This might be slow for a very long input if done naively. But let's try a direct implementation first.
    # Optimization might be needed if input is large.
    #
    # Steps:
    # 1. Find the leftmost '.' that still has at least one file block to the right.
    # 2. Find the rightmost file block on the entire disk (or to the right of that '.'?), the problem states:
    #    "moving from the end of the disk to fill the leftmost free block"
    #    So we take the rightmost file block (the last non-'.' character from the end).
    # 3. Move that file block into the '.' position.
    # 4. Mark the old file block position as '.'.
    #
    # Repeat until no '.' is found that has any file block to its right (meaning all '.' are at the end).

    # Check if we need to do many moves:
    # Let's implement the described loop.

    while True:
        # Find the leftmost '.' that has a file block to its right
        # i.e. find '.' at index i, and ensure there's at least one file block (not '.') in disk[i+1:]
        try:
            gap_index = next(i for i, ch in enumerate(disk) if ch == '.' and any(c != '.' for c in disk[i+1:]))
        except StopIteration:
            # No such gap found, we are done
            break

        # Find the rightmost file block
        # That is the highest index j where disk[j] != '.'
        for j in range(len(disk)-1, -1, -1):
            if disk[j] != '.':
                # Move this block to gap_index
                disk[gap_index] = disk[j]
                disk[j] = '.'
                break
        # Loop again to see if more moves are needed

    # Now disk is compacted.

    # Compute the checksum:
    # sum of (position * file_id) for every file block
    # file_id was stored as a string, so convert back to int to multiply
    checksum = 0
    for pos, ch in enumerate(disk):
        if ch != '.':
            checksum += pos * int(ch)

    print(checksum)


if __name__ == "__main__":
    main()


6430446922192


## Part 2

In [2]:
def main():
    # Read the input line
    with open('input.txt', 'r') as f:
        line = f.readline().strip()

    # Parse the input into a disk layout similar to part one.
    digits = list(map(int, line))
    file_lengths = []
    free_lengths = []
    file_next = True
    for d in digits:
        if file_next:
            file_lengths.append(d)
        else:
            free_lengths.append(d)
        file_next = not file_next
    if len(free_lengths) < len(file_lengths):
        free_lengths.append(0)

    disk = []
    file_id = 0
    # Construct the disk layout with file IDs and '.' for free
    for f_len, fs_len in zip(file_lengths, free_lengths):
        disk.extend([str(file_id)] * f_len)
        disk.extend(['.'] * fs_len)
        file_id += 1

    # Now we have a disk layout: a list of strings, each either '.' or the file_id as string.
    # We need to move whole files, starting from the highest file ID down to the lowest.
    # Procedure:
    #   For each file (in decreasing ID):
    #       1. Find all contiguous blocks of '.' to the left of the start of that file.
    #       2. If any of those free blocks can fit the entire file, move the file there.
    #          If multiple possible places, we should choose the leftmost free span that can hold the file.
    #          (The problem doesn't explicitly say how to choose if multiple fits are possible,
    #           but logically, we should pick the earliest suitable free space from left to right.)
    #       3. If no suitable free space is found, do not move the file.

    # Identify all files: 
    # We'll need for each file_id: the indices of its blocks.
    # We'll do a function to find file positions whenever we need them, because file positions can change.

    def get_file_blocks(fid):
        return [i for i, ch in enumerate(disk) if ch == str(fid)]

    # To find free spaces to the left of the file start:
    # For a given file, we find its current positions, let start = min(file_positions).
    # Then consider the region disk[0:start] and find all contiguous '.' runs.
    # Check if any run length >= file_length.

    # For placing the file, we must find the leftmost suitable free run and move the file there.
    # Moving the file involves:
    #   - Determine the starting position of the chosen free run
    #   - Overwrite that run with the file_id blocks
    #   - Free the old positions of the file by replacing with '.'
    # The file must remain contiguous after the move.

    max_file_id = file_id - 1
    for fid in range(max_file_id, -1, -1):
        file_blocks = get_file_blocks(fid)
        if not file_blocks:
            # File not found? (should not happen, but just in case)
            continue
        file_length = len(file_blocks)
        file_start = min(file_blocks)  # leftmost position of the file
        # Identify free runs to the left of file_start
        left_region = disk[:file_start]
        # Find contiguous '.' runs in left_region
        free_runs = []
        run_start = None
        for i, ch in enumerate(left_region):
            if ch == '.':
                if run_start is None:
                    run_start = i
            else:
                if run_start is not None:
                    free_runs.append((run_start, i - 1))
                    run_start = None
        # If ended in a run:
        if run_start is not None:
            free_runs.append((run_start, len(left_region) - 1))

        # Check if any run can hold the entire file_length
        # Choose the leftmost run that fits
        candidate = None
        for start_idx, end_idx in free_runs:
            run_len = end_idx - start_idx + 1
            if run_len >= file_length:
                candidate = (start_idx, start_idx + file_length - 1)  # exact placement range
                break

        if candidate:
            new_start, new_end = candidate
            # Move the file there
            # Overwrite the candidate run positions with file_id blocks
            for i in range(new_start, new_end + 1):
                disk[i] = str(fid)
            # Free the old positions
            for i in file_blocks:
                disk[i] = '.'

    # After attempting all moves, compute the checksum:
    # sum of position * file_id for all file blocks.
    checksum = 0
    for pos, ch in enumerate(disk):
        if ch != '.':
            checksum += pos * int(ch)

    print(checksum)


if __name__ == "__main__":
    main()


6460170593016
